In [1]:
!pip install -e .
!pip install spacy
!python3 -m spacy download en_core_web_lg
!pip install pandas
!pip install openpyxl


Obtaining file:///Users/rajashreedahal/Desktop/Metadata_Finetuning/larch
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done


  Building editable for larch (pyproject.toml) ... done
  Created wheel for larch: filename=larch-0.0.2a0-0.editable-py3-none-any.whl size=14420 sha256=b4a1ca9cb0967964956c75d05e47005db812d68f346b2a630081a1defaaa3c95
  Stored in directory: /private/var/folders/m4/9_ydsvdn3b3d84rntrrsxm0h0000gn/T/pip-ephem-wheel-cache-ojilh19z/wheels/3a/98/39/46f8f1efc02842106044493327f82ab1f2c43a0a7f389a1df9
Successfully built larch
  Attempting uninstall: larch
    Found existing installation: larch 0.0.2a0
    Uninstalling larch-0.0.2a0:
      Successfully uninstalled larch-0.0.2a0

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 14.3 MB/s eta 0:00:00m eta 0:00:010:00:01



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [32]:
import pandas as pd
from larch.metadata import InstructorBasedOpenAIMetadataExtractor
from larch.metadata.validators import WhitelistBasedMetadataValidator
from larch.processors import PIIRemover, TextProcessingPipeline
from larch.utils import load_whitelist

text_processor = TextProcessingPipeline(
    lambda x: re.sub(r"\$(?=\w|\n|\()", " ", x).strip(),
    lambda x: re.sub(r"\)(?=\w|\n|\()", " ", x).strip(),
    lambda x: re.sub(r"\#(?=\w|\n|\()", " ", x).strip(),
    lambda x: x.replace("\t", " ").replace("!", " ").strip(),
    PIIRemover()
)
import re
import requests
from bs4 import BeautifulSoup

from pydantic import BaseModel,constr,Field
from typing import Literal

class Environmental_Justice(BaseModel):
    Indicators: Literal["Disasters","Human Dimensions","Food Availability","Health & Air Quality","Water Availability", "Extreme Heat","Urban Flooding","Climate"]
    Description: str
    Geographic_Coverage: str =Field(pattern=r'')
    Format: str
    Spatial_Resolution: str = Field(pattern=r'^(\d+(\.\d+)? [a-zA-Z]+|varies|N/A)$')
    Temporal_Resolution: str = Field(pattern=r'^(\d+(\.\d+)? [a-zA-Z]+|N/A|varies|weekly|monthly|daily|yearly|varies-multiple datasets included|Daily < Weekly|Hourly < Daily|Weekly < Monthly|Monthly < Yearly|1 minute)$')
    Temporal_Extent: str = Field(pattern=r'^(\d{4}-\d{2}-\d{2} to present|present|\d{4}-\d{2}-\d{2} (?:to|until) present|\d{4}-\d{2}-\d{2} to \d{4}-\d{2}-\d{2}|varies(?:- multiple datasets (?:included|available))?)$')
    Latency: str = Field(pattern=r'^\d+(\s*(Day|days|months|NRT))?$')
    Project: str
    Data_Visualization: bool


    
schema = Environmental_Justice
sheet_names=["Water Availability - Cleaned", "Disasters (Disaster Recovery) -", 
 "Food Availability - Cleaned", "Human Dimensions - Cleaned", "Urban Flooding - Cleaned",
 "Extreme Heat - CIP","Climate (Climate Change) - CIP","Health and Air Quality - CIP"]
sheet_names=sheet_names[4:]
# sheet_names=sheet_names[1:]
# sheet_names=["Water Availability - Cleaned"]
# sheet_names=["Disasters (Disaster Recovery) -"]
# sheet_names=sheet_names[6:]
# sheet_names=["Climate (Climate Change) - CIP"]
sheet_names=["Urban Flooding - Cleaned"]
used_urls=[]
for each_sheet in sheet_names:
    mapper = pd.read_excel("nasa_esds.xlsx", sheet_name=[each_sheet])
    res = {}
    url_lists=[]
    for field_key, value_df in mapper.items():
        res[field_key] = {}
        cols = value_df.columns
        for i, row in value_df.iterrows():
            source_link_col = [col for col in cols if col == "Source/Link"]
            alternate_vals = filter(None, row[source_link_col])
            alternate_vals=list(alternate_vals)
            if alternate_vals and str(alternate_vals[0]).startswith("http"):
                url_lists.append(alternate_vals[0])

    url_content=[]
    urls=[]
    count=0
    url_lists=["https://dx.doi.org/10.5067/ECTSD-MSL44","https://dx.doi.org/10.5067/ECTSM-MSL44","https://dx.doi.org/10.5067/GMSLM-TJ151",
 "https://dx.doi.org/10.5067/GMSLT-FJPL1","https://cmr.earthdata.nasa.gov/search/concepts/C2157848116-PODAAC.html",
 'https://cmr.earthdata.nasa.gov/search/concepts/C2205556193-POCLOUD.html',
 'https://dx.doi.org/10.5067/ALTTS-TJA51',
 'https://dx.doi.org/10.5067/ALTCY-TJA51',
 'https://gpm.nasa.gov/data/directory']
    for url in url_lists:
        response=requests.get(url)
        html_page = response.text
        soup = BeautifulSoup(html_page, "html.parser")
        text=soup.get_text()
        text = re.sub(r'[\t\n\r\f\v]+', '', text)
        text=re.sub(r'[^\w\s]', '', text)
        if url not in used_urls:
            url_content.append(text)
            urls.append(url)
        used_urls.append(url)


    import json

    metadata_extractor = InstructorBasedOpenAIMetadataExtractor(
        model="gpt-4",
        schema=schema,
        preprocessor=text_processor,
        debug=True,
    )

    response_result={}
    for enum,text in enumerate(url_content):
        split=text.split()
        if len(split)>1000:
            text=" ".join(split[0:1000])
        metadata = metadata_extractor(text)
        response=json.loads((metadata['choices'][0]['message']["function_call"]["arguments"]))
        response_result[urls[enum]]=response
        print(enum,len(text.split()))
    with open("extractions/gpt4_prompt2/"+each_sheet+".json",'w') as file:
            json.dump(response_result,file,indent=4)


The schema is <class '__main__.Environmental_Justice'>


2024-02-09 13:15:03.046 | DEBUG    | larch.metadata.extractors_openai:_extract:79 - nchars=10143
Text :: ECCO Global Mean Sea Level  Daily Mean Version 4 Release 4  PODAAC  JPL  NASA Search Type WebsiteDataSearch   Jet Propulsion LaboratoryCalifornia Institute of Technology PODAACFollow UsNavigation   Search Type WebsiteDataSearch HomeFind DataScience DisciplinesOcean Cryosphere Terrestrial Hydrosphere MeasurementsGlaciersIce SheetsGravityGravitational FieldOcean CirculationOcean Heat BudgetOcean Surface TopographyOcean TemperatureOcean WavesOcean WindsSalinityDensitySea IceSurface WaterMissionsADEOSIIAQUAAQUARIUSSACDCOWVRTEMPESTCYGNSSECCOGEOS3GHRSSTGRACEGRACEFOISSRAPIDSCAT JASON 1JASON 3MEASURESCCMPMEASURESMURMEASURESPRESWOTNASASSHNSCATOMGOPERAOSTMJASON 2QUIKSCATSMODESNPPSASSIESEASATSENTINEL6SMAPSPURSSWOTTERRATOPEXPOSEIDONAccess DataRESOURCESCloud Data UsersData ProvidersData UsersAboutABOUT UsABOUT PODAACData Use and Citation PolicyMEASUREMENTSGlaciersIce SheetsGravityGravitational F

the system prompt is Extract metadata and entities details accurately from my requests.Metadata extraction for Indicators should clearly be one of the element in the list [“Disasters”,“Human Dimensions”,“Food Availability”,“Health & Air Quality”,“Water Availability”, “Extreme Heat”,“Urban Flooding”,“Climate]. Do not create new element for indicators Make sure the following fields follow the following regex patternSpatial_Resolution: ^(\d+(\.\d+)? [a-zA-Z]+|varies|N/A)$Temporal_Resolution: ^(\d+(\.\d+)? [a-zA-Z]+|N/A|varies|weekly|monthly|daily|yearly|varies-multiple datasets included|Daily < Weekly|Hourly < Daily|Weekly < Monthly|Monthly < Yearly|1 minute)$Temporal_Extent: ^(\d{4}-\d{2}-\d{2} to present|present|\d{4}-\d{2}-\d{2} (?:to|until) present|\d{4}-\d{2}-\d{2} to \d{4}-\d{2}-\d{2}|varies(?:- multiple datasets (?:included|available))?)$
The messages is [{'role': 'system', 'content': 'Extract metadata and entities details accurately from my requests.Metadata extraction for Indicat

0 971


2024-02-09 13:15:16.829 | DEBUG    | larch.metadata.extractors_openai:_extract:79 - nchars=10195
Text :: ECCO Global Mean Sea Level  Monthly Mean Version 4 Release 4  PODAAC  JPL  NASA Search Type WebsiteDataSearch   Jet Propulsion LaboratoryCalifornia Institute of Technology PODAACFollow UsNavigation   Search Type WebsiteDataSearch HomeFind DataScience DisciplinesOcean Cryosphere Terrestrial Hydrosphere MeasurementsGlaciersIce SheetsGravityGravitational FieldOcean CirculationOcean Heat BudgetOcean Surface TopographyOcean TemperatureOcean WavesOcean WindsSalinityDensitySea IceSurface WaterMissionsADEOSIIAQUAAQUARIUSSACDCOWVRTEMPESTCYGNSSECCOGEOS3GHRSSTGRACEGRACEFOISSRAPIDSCAT JASON 1JASON 3MEASURESCCMPMEASURESMURMEASURESPRESWOTNASASSHNSCATOMGOPERAOSTMJASON 2QUIKSCATSMODESNPPSASSIESEASATSENTINEL6SMAPSPURSSWOTTERRATOPEXPOSEIDONAccess DataRESOURCESCloud Data UsersData ProvidersData UsersAboutABOUT UsABOUT PODAACData Use and Citation PolicyMEASUREMENTSGlaciersIce SheetsGravityGravitational

...

2024-02-09 13:16:47.403 | DEBUG    | larch.metadata.extractors_openai:_extract:79 - nchars=9789
Text :: Integrated MultiMission Ocean Altimeter Data for Climate Research Version 51 PODAAC JPL NASA Search Type WebsiteDataSearch Jet Propulsion LaboratoryCalifornia Institute of Technology PODAACFollow UsNavigation Search Type WebsiteDataSearch HomeFind DataScience DisciplinesOcean Cryosphere Terrestrial Hydrosphere MeasurementsGlaciersIce SheetsGravityGravitational FieldOcean CirculationOcean Heat BudgetOcean Surface TopographyOcean TemperatureOcean WavesOcean WindsSalinityDensitySea IceSurface WaterMissionsADEOSIIAQUAAQUARIUSSACDCOWVRTEMPESTCYGNSSECCOGEOS3GHRSSTGRACEGRACEFOISSRAPIDSCAT JASON 1JASON 3MEASURESCCMPMEASURESMURMEASURESPRESWOTNASASSHNSCATOMGOPERAOSTMJASON 2QUIKSCATSMODESNPPSASSIESEASATSENTINEL6SMAPSPURSSWOTTERRATOPEXPOSEIDONAccess DataRESOURCESCloud Data UsersData ProvidersData UsersAboutABOUT UsABOUT PODAACData Use and Citation PolicyMEASUREMENTSGlaciersIce SheetsGravityGravi

6 1000
the system prompt is Extract metadata and entities details accurately from my requests.Metadata extraction for Indicators should clearly be one of the element in the list [“Disasters”,“Human Dimensions”,“Food Availability”,“Health & Air Quality”,“Water Availability”, “Extreme Heat”,“Urban Flooding”,“Climate]. Do not create new element for indicators Make sure the following fields follow the following regex patternSpatial_Resolution: ^(\d+(\.\d+)? [a-zA-Z]+|varies|N/A)$Temporal_Resolution: ^(\d+(\.\d+)? [a-zA-Z]+|N/A|varies|weekly|monthly|daily|yearly|varies-multiple datasets included|Daily < Weekly|Hourly < Daily|Weekly < Monthly|Monthly < Yearly|1 minute)$Temporal_Extent: ^(\d{4}-\d{2}-\d{2} to present|present|\d{4}-\d{2}-\d{2} (?:to|until) present|\d{4}-\d{2}-\d{2} to \d{4}-\d{2}-\d{2}|varies(?:- multiple datasets (?:included|available))?)$
The messages is [{'role': 'system', 'content': 'Extract metadata and entities details accurately from my requests.Metadata extraction for 

2024-02-09 13:16:59.035 | DEBUG    | larch.metadata.extractors_openai:_extract:79 - nchars=8513
Text :: Precipitation Data Directory NASA Global Precipitation Measurement Mission Skip to main content GLOBALPRECIPITATIONMEASUREMENTSEARCH CONTACT MissionsGPMGPM Core ObservatoryGPM Microwave Imager GMIDualfrequency Precipitation Radar DPRGPM ConstellationLaunchTRMMSpacecraft and InstrumentsExtreme Weather NewsArticlesDataData DirectoryData SourcesData NewsData FAQData PolicyTrainingTutorialsGround Validation DataIMERGPrecipitation ClimatologySeasonal Precipitation VariationsVisualizationIMERG Global ViewerNASA WorldviewPrecipitation Applications ViewerArticlesApplicationsEcologyWater AgricultureEnergyWeatherExtreme Weather NewsHealthDisease InitiativeDisastersLandslidesWhos Using GPM DataApplications HighlightsArticlesScienceResearch TopicsStorm Structure and Mesoscale DynamicsThe Global Water CycleClimate AnalysisPrecipitation MicrophysicsGround ValidationField CampaignsGround and Airbor

7 1000
the system prompt is Extract metadata and entities details accurately from my requests.Metadata extraction for Indicators should clearly be one of the element in the list [“Disasters”,“Human Dimensions”,“Food Availability”,“Health & Air Quality”,“Water Availability”, “Extreme Heat”,“Urban Flooding”,“Climate]. Do not create new element for indicators Make sure the following fields follow the following regex patternSpatial_Resolution: ^(\d+(\.\d+)? [a-zA-Z]+|varies|N/A)$Temporal_Resolution: ^(\d+(\.\d+)? [a-zA-Z]+|N/A|varies|weekly|monthly|daily|yearly|varies-multiple datasets included|Daily < Weekly|Hourly < Daily|Weekly < Monthly|Monthly < Yearly|1 minute)$Temporal_Extent: ^(\d{4}-\d{2}-\d{2} to present|present|\d{4}-\d{2}-\d{2} (?:to|until) present|\d{4}-\d{2}-\d{2} to \d{4}-\d{2}-\d{2}|varies(?:- multiple datasets (?:included|available))?)$
The messages is [{'role': 'system', 'content': 'Extract metadata and entities details accurately from my requests.Metadata extraction for 

8 1000


In [2]:
import math
import pandas as pd
import json

# Evalutation metrics for sample data sheet

In [3]:
sheet_names=["Water Availability - Cleaned", "Disasters (Disaster Recovery) -", 
 "Food Availability - Cleaned", "Human Dimensions - Cleaned", "Urban Flooding - Cleaned",
 "Extreme Heat - CIP","Health and Air Quality - CIP", "Climate (Climate Change) - CIP"]
count=0
true_data = {}
repeated_rows=[]
for sheet_name in sheet_names:
    mapper = pd.read_excel("nasa_esds.xlsx", sheet_name=[sheet_name])
    for field_key, value_df in mapper.items():
        cols = value_df.columns
        for i, row in value_df.iterrows():
            source_link_col = ["Source/Link","Indicators                     (Select from drop-down list)","Description","Description Simplified","Geographic Coverage","Format","Spatial Resolution","Spatial Resolution (standard)","Temporal Resolution","Temporal Extent","Latency","Project","Data Visualization"]
            alternate_vals=row[source_link_col]
            alternate_vals=list(alternate_vals)
            source_link_col[1]="Indicators"
            alternate_values=alternate_vals[1:]
            value={}
            count=count+1
            for enum,i in enumerate(source_link_col[1:]):
                if i=="Data Visualization":
                    if isinstance(alternate_values[enum],str):
                        value[i]=True
                    else:
                        value[i]=False
                else:
                    value[i]=alternate_values[enum]
            if alternate_vals[0] in true_data.keys():
                repeated_rows.append(alternate_vals[0])
            true_data[alternate_vals[0]]=value

            
        

print("The count is",count)
print(len(true_data))  #(278-152 duplicates)

The count is 278
152


In [4]:
true_data

{'https://doi.org/10.5067/MEaSUREs/GFSAD/GFSAD30AFCE.001': {'Indicators': 'Food Availability',
  'Description': 'The NASA Making Earth System Data Records for Use in Research Environments (MEaSUREs) (https://earthdata.nasa.gov/community/community-data-system-programs/measures-projects) Global Food Security-support Analysis Data (GFSAD) data product provides cropland extent data over the continent of Africa for nominal year 2015 at 30 meter resolution (GFSAD30AFCE). The monitoring of global cropland extent is critical for policymaking and provides important baseline data that are used in many agricultural cropland studies pertaining to water sustainability and food security. 2013-01-01 to 2016-06-30\n',
  'Description Simplified': 'The Global Food Security-support Analysis Data (GFSAD) Cropland Extent 2015 Africa 30 m V001 dataset provides cropland extent data over Africa for the year 2015. These data may be useful in agricultural cropland studies related to water sustainability and foo

In [5]:
import json
pred_data={}
for sheet_name in sheet_names:
    with open(f"extractions/gpt4_prompt2/{sheet_name}.json",'r') as file:
        data=json.load(file)
    pred_data.update(data)


In [6]:
# !pip install sentence_transformers
from difflib import SequenceMatcher
import numpy as np
import re 
import torch
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim


encoder = SentenceTransformer('all-mpnet-base-v2').to('cpu')

# Evaluation Metric: Exact Match  #Indicators, temporal resolution, temporal extent,
def evaluate_exact_match(predicted, ground_truth):
    return int(predicted.lower() == ground_truth.lower())

# Evaluation Metric: description, project, geographic measure
def evaluate_similarity_or_edit_distance(predicted, ground_truth1,ground_truth2,encoder):
    if (type(ground_truth1)==float and type(predicted)==float) or (type(ground_truth2)==float and type(predicted)==float):
        return 1
    elif type(ground_truth1)==float or type(ground_truth2)==float or type(predicted)==float:
        return 0
        
    with torch.no_grad():
        pred_embeddings = encoder.encode(predicted, show_progress_bar=False)
        truth1_embeddings = encoder.encode(ground_truth1, show_progress_bar=False)
        truth2_embeddings = encoder.encode(ground_truth2, show_progress_bar=False)
    similarity1 = cos_sim(pred_embeddings, truth1_embeddings).numpy()[0][0]
    similarity2 = cos_sim(pred_embeddings, truth2_embeddings).numpy() [0][0]
    similarity=max(similarity1,similarity2)
    if similarity>0.7:
        return 1
    else:
        return 0
    
def evaluate_project(predicted, ground_truth,encoder):
    if type(predicted)==float and type(ground_truth)==float:
        return 1
    elif type(ground_truth)==float:
        return 0
    ground_truth=ground_truth.split("-",1)
    with torch.no_grad():
        pred_embeddings = encoder.encode(predicted, show_progress_bar=False)
        truth1_embeddings = encoder.encode(ground_truth[0], show_progress_bar=False)
        similarity = cos_sim(pred_embeddings, truth1_embeddings).numpy()[0][0]
        if len(ground_truth)>1:
            truth2_embeddings = encoder.encode(ground_truth[1], show_progress_bar=False)
            similarity1 = cos_sim(pred_embeddings, truth2_embeddings).numpy()[0][0]
            similarity=max(similarity,similarity1)
    if similarity>0.55:
        return 1
    else:
        return 0


# Evaluation Metric: Exact Match or at least one match if multiple
def evaluate_format(predicted, ground_truth):
    predicted = np.nan if predicted=="N/A" else predicted
    if type(predicted)==float and type(ground_truth)==float:
        return 1
    elif type(predicted)==float or type(ground_truth)==float:
        return 0
    predicted=predicted.lower()
    ground_truth=ground_truth.lower()
    if predicted==ground_truth:
        return 1
    predicted_formats = set(predicted.split(","))
    ground_truth_formats = set(ground_truth.split(","))
    if predicted_formats.intersection(ground_truth_formats):
        return 1
    return 0

def evaluate_spatial_resolution(predicted, ground_truth1,ground_truth2):
    predicted = np.nan if predicted=="N/A" else predicted
    if (type(predicted)==float and type(ground_truth1)==float) or (type(predicted)==float and type(ground_truth2)==float):
        return 1
    elif (type(ground_truth1)==float or type(ground_truth2)==float):
        return 0
    elif type(predicted)==float:
        return 0
    predicted=predicted.lower()
    ground_truth1=ground_truth1.lower()
    ground_truth2=ground_truth2.lower()
    if (predicted==ground_truth1) or (predicted==ground_truth2):
        return 1
    predicted = re.sub(r'[a-zA-Z\(\)\[\]]', '', predicted).strip()
    ground_truth1=re.sub(r'[a-zA-Z\(\)\[\]]', '', ground_truth1).strip()
    ground_truth2=re.sub(r'[a-zA-Z\(\)\[\]]', '', ground_truth2).strip()
    if not (ground_truth1 or ground_truth2) and predicted.find(',') != -1:
        return 1
    ground_truth1=re.sub(r'[^\d.]', '', ground_truth1)
    ground_truth2=re.sub(r'[^\d.]', '', ground_truth2)
    predicted=re.sub(r'[^\d.]', '', predicted)
    if predicted==ground_truth1 or predicted==ground_truth2:
        return 1
    if predicted and ((ground_truth1 and (predicted.replace(" ","")==ground_truth1.replace(" ",""))) or (ground_truth2 and (predicted.replace(" ","")==ground_truth2.replace(" ","")))):
        return 1
    return 0


def evaluate_latency(predicted, ground_truth):
    predicted = np.nan if predicted=="N/A" else predicted
    if type(predicted)==float and type(ground_truth)==float:
        return 1
    if type(predicted)==float or type(ground_truth)==float:
        return 0
    predicted=predicted.lower()
    ground_truth=ground_truth.lower()
    if (predicted==ground_truth):
        return 1
    predicted = re.sub(r'[a-zA-Z]', '', predicted)
    predicted=re.sub(r'[^a-zA-Z0-9]','',predicted)
    ground_truth=re.sub(r'[a-zA-Z]', '', ground_truth)
    ground_truth=re.sub(r'[^a-zA-Z0-9]','',ground_truth)
    if not ground_truth and not predicted:
        return 1
    if not ground_truth or not predicted:
        return 0
    if (int(predicted)==int(ground_truth)):
        return 1
    return 0


def evaluate_temporal_resolution_extent(predicted,ground_truth):
    predicted = np.nan if predicted=="N/A" else predicted
    ground_truth=np.nan if (type(ground_truth)==str and ground_truth.lower()=="nan") else ground_truth
    if type(predicted)==float and type(ground_truth)==float:
        return 1
    if type(predicted)==float or type(ground_truth)==float:
        return 0
    predicted=predicted.lower()
    ground_truth=str(ground_truth).lower()
    if predicted==ground_truth:
        return 1
    predicted = ''.join(re.findall(r'\w', predicted))
    ground_truth=''.join(re.findall(r'\w', ground_truth))
    if predicted==ground_truth:
        return 1 
    return 0

def evaluate_data_visualization_indicators(predicted,ground_truth):
    if predicted==ground_truth:
        return 1
    return 0
    


In [7]:
len(pred_data)

147

In [8]:
metrics={}
for key in true_data.keys():
    reject_key=[np.nan, 'DBF', 'Facebook Data For Good High Resolution Population Density Maps']
    if key not in reject_key:
        true_source=true_data[key]
        try:
            pred_source=pred_data[key]
        except:
            pred_source=pred_data[key.strip()]
        for each_key in pred_source.keys():
            if each_key=="Indicators":
                result=evaluate_exact_match(pred_source[each_key],true_source[each_key])
                if each_key not in metrics.keys():
                        metrics[each_key]=[result]
                else:
                    metrics[each_key].append(result)
            elif each_key=="Description":
                result=evaluate_similarity_or_edit_distance(pred_source[each_key], true_source[each_key],true_source["Description Simplified"],encoder)
                if each_key not in metrics.keys():
                    metrics[each_key]=[result]
                else:
                    metrics[each_key].append(result)
            elif each_key=="Geographic_Coverage":
                evaluate_format(pred_source[each_key],true_source["Geographic Coverage"])
                if each_key not in metrics.keys():
                    metrics[each_key]=[result]
                else:
                    metrics[each_key].append(result) 
            elif each_key=="Format":
                result=evaluate_format(pred_source[each_key],true_source[each_key])
                if each_key not in metrics.keys():
                    metrics[each_key]=[result]
                else:
                    metrics[each_key].append(result)
            elif each_key=="Spatial_Resolution":
                result=evaluate_spatial_resolution(pred_source[each_key],true_source["Spatial Resolution"],true_source["Spatial Resolution (standard)"])
                if each_key not in metrics.keys():
                    metrics[each_key]=[result]
                else:
                    metrics[each_key].append(result)
            elif each_key=="Temporal_Resolution":
                result=evaluate_temporal_resolution_extent(pred_source[each_key],true_source["Temporal Resolution"])
                if each_key not in metrics.keys():
                    metrics[each_key]=[result]
                else:
                    metrics[each_key].append(result)
            elif each_key=="Temporal_Extent":
                result=evaluate_temporal_resolution_extent(pred_source[each_key],true_source["Temporal Extent"])
                if each_key not in metrics.keys():
                    metrics[each_key]=[result]
                else:
                    metrics[each_key].append(result)
            elif each_key=="Latency":
                result=evaluate_latency(pred_source[each_key],true_source["Latency"])
                if each_key not in metrics.keys():
                    metrics[each_key]=[result]
                else:
                    metrics[each_key].append(result)
            elif each_key=="Project":
                result=evaluate_project(pred_source[each_key], true_source[each_key],encoder)
                if each_key not in metrics.keys():
                    metrics[each_key]=[result]
                else:
                    metrics[each_key].append(result)
            elif each_key=="Data_Visualization":
                result=evaluate_data_visualization_indicators(pred_source[each_key], true_source["Data Visualization"])
                if each_key not in metrics.keys():
                    metrics[each_key]=[result]
                else:
                    metrics[each_key].append(result)

In [ ]:
predictions=metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
aspect_metrics = {}
for aspect, preds in predictions.items():
    accuracy = accuracy_score(preds, [1]*len(preds))  # Assuming ground truth is always 1
    precision = precision_score(preds, [1]*len(preds))
    recall = recall_score(preds, [1]*len(preds))
    f1 = f1_score(preds, [1]*len(preds))
    aspect_metrics[aspect] = {'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1-score': f1}

# Print metrics
for aspect, metrics_ in aspect_metrics.items():
    print(aspect)
    for metric, value in metrics_.items():
        print(f"{metric}: {value:.2f}")
    print("\n")

In [9]:
#result for gptt 4 (2) based prompt

Indicators
Accuracy: 0.26
Precision: 0.26
Recall: 1.00
F1-score: 0.41


Description
Accuracy: 0.63
Precision: 0.63
Recall: 1.00
F1-score: 0.77


Geographic_Coverage
Accuracy: 0.63
Precision: 0.63
Recall: 1.00
F1-score: 0.77


Format
Accuracy: 0.11
Precision: 0.11
Recall: 1.00
F1-score: 0.19


Spatial_Resolution
Accuracy: 0.35
Precision: 0.35
Recall: 1.00
F1-score: 0.52


Temporal_Resolution
Accuracy: 0.30
Precision: 0.30
Recall: 1.00
F1-score: 0.46


Temporal_Extent
Accuracy: 0.30
Precision: 0.30
Recall: 1.00
F1-score: 0.46


Latency
Accuracy: 0.76
Precision: 0.76
Recall: 1.00
F1-score: 0.86


Project
Accuracy: 0.38
Precision: 0.38
Recall: 1.00
F1-score: 0.55


Data_Visualization
Accuracy: 0.66
Precision: 0.66
Recall: 1.00
F1-score: 0.80




In [14]:
#Output for gpt 3.5 prompt2

Indicators
Accuracy: 0.26
Precision: 0.26
Recall: 1.00
F1-score: 0.41


Description
Accuracy: 0.62
Precision: 0.62
Recall: 1.00
F1-score: 0.77


Geographic_Coverage
Accuracy: 0.62
Precision: 0.62
Recall: 1.00
F1-score: 0.77


Format
Accuracy: 0.11
Precision: 0.11
Recall: 1.00
F1-score: 0.20


Spatial_Resolution
Accuracy: 0.38
Precision: 0.38
Recall: 1.00
F1-score: 0.55


Temporal_Resolution
Accuracy: 0.38
Precision: 0.38
Recall: 1.00
F1-score: 0.55


Temporal_Extent
Accuracy: 0.18
Precision: 0.18
Recall: 1.00
F1-score: 0.31


Latency
Accuracy: 0.78
Precision: 0.78
Recall: 1.00
F1-score: 0.87


Project
Accuracy: 0.23
Precision: 0.23
Recall: 1.00
F1-score: 0.37


Data_Visualization
Accuracy: 0.58
Precision: 0.58
Recall: 1.00
F1-score: 0.74




# Output for GPT 3.5 prompt

In [15]:
values=pred_data.values()
dist=[]
for each_value in values:
    if each_value:
        ind=each_value['Indicators']
        if ind not in dist:
            dist.append(ind)
                            
print(dist)

['Food Availability', 'Climate', 'Water Availability', 'Disasters', 'Human Dimensions', 'Extreme Heat', 'Urban Flooding', 'Poverty Mapping', 'Urban Land', 'Health & Air Quality', 'Population', 'Land Use and Land Cover', 'Poverty', 'Economic Data', 'Infrastructure', 'Population Data', 'Air Quality']


# whitelist validator

In [1]:
import pandas as pd
from larch.metadata import InstructorBasedOpenAIMetadataExtractor
from larch.metadata.validators import WhitelistBasedMetadataValidator
from larch.processors import PIIRemover, TextProcessingPipeline
from larch.utils import load_whitelist
from larch.processors import NonAlphaNumericRemover

text_processor = TextProcessingPipeline(
    lambda x: re.sub(r"\$(?=\w|\n|\()", " ", x).strip(),
    lambda x: re.sub(r"\)(?=\w|\n|\()", " ", x).strip(),
    lambda x: re.sub(r"\#(?=\w|\n|\()", " ", x).strip(),
    lambda x: x.replace("\t", " ").replace("!", " ").strip(),
    PIIRemover()
)
import re
import requests
from bs4 import BeautifulSoup

from pydantic import BaseModel,constr,Field
from typing import Literal
class Environmental_Justice(BaseModel):
    Indicators: Literal["Disasters","Human Dimensions","Food Availability","Health & Air Quality","Water Availability", "Extreme Heat","Urban Flooding","Climate"]
    Description: str
    Geographic_Coverage: str =Field(pattern=r'')
    Format: str
    Spatial_Resolution: str = Field(pattern=r'^(\d+(\.\d+)? [a-zA-Z]+|varies|N/A)$')
    Temporal_Resolution: str = Field(pattern=r'^(\d+(\.\d+)? [a-zA-Z]+|N/A|varies|weekly|monthly|daily|yearly|varies-multiple datasets included|Daily < Weekly|Hourly < Daily|Weekly < Monthly|Monthly < Yearly|1 minute)$')
    Temporal_Extent: str = Field(pattern=r'^(\d{4}-\d{2}-\d{2} to present|present|\d{4}-\d{2}-\d{2} (?:to|until) present|\d{4}-\d{2}-\d{2} to \d{4}-\d{2}-\d{2}|varies(?:- multiple datasets (?:included|available))?)$')
    Latency: str = Field(pattern=r'^\d+(\s*(Day|days|months|NRT))?$')
    Project: str
    Data_Visualization: bool

        
schema=Environmental_Justice

whitelists={"Indicators":["Disasters","Human Dimensions","Food Availability","Health & Air Quality","Water Availability", "Extreme Heat","Urban Flooding","Climate"]}
validator = WhitelistBasedMetadataValidator(
                whitelists=whitelists,
                fuzzy_threshold=0.90,
                fuzzy_scorer=0.9,
                text_processor=NonAlphaNumericRemover(),
                debug=False,
            )

metadata_extractor = InstructorBasedOpenAIMetadataExtractor(
    model="gpt-3.5-turbo-0125",
    schema=schema,
    preprocessor=text_processor,
    debug=True,
)
import requests
# text="The U.S. Social Vulnerability Index Grids identifies socially vulnerable populations at higher risk due to four main factors: socioeconomic status, household composition & disability, minority status & language, and housing type & transportation. These data are produced by the CDC (Centers for Disease Control and Prevention based on census data from the years 200, 2010, 2014, 2016, and 2018."
# text="NASA's Socioeconomic Data and Applications Center (SEDAC) offers global annual gridded PM 2.5 data for various years. This data  was aggregated to common global boundaries in order to help us see what air quality is like around the world."
text="This dataset provides daily-averaged global mean sea level from the ECCO Version 4 Release 4 (V4r4) ocean and sea-ice state estimate. Estimating the Circulation and Climate of the Ocean (ECCO) ocean and sea-ice state estimates are dynamically and kinematically-consistent reconstructions of the three-dimensional time-evolving ocean, sea-ice, and surface atmospheric states. ECCO V4r4 is a free-running solution of the 1-degree global configuration of the MIT general circulation model (MITgcm) that has been fit to observations in a least-squares sense."
response=requests.get("https://dx.doi.org/10.5067/GMSLM-TJ151")
html_page = response.text
soup = BeautifulSoup(html_page, "html.parser")
text=soup.get_text()
text = re.sub(r'[\t\n\r\f\v]+', '', text)
text=re.sub(r'[^\w\s]', '', text)
metadata = metadata_extractor(text)
metadata = validator(metadata)


spacy is loaded
The schema is <class '__main__.Environmental_Justice'>


2024-02-09 10:55:54.522 | DEBUG    | larch.metadata.extractors_openai:_extract:79 - nchars=14528
Text :: Global Mean Sea Level Trend from Integrated MultiMission Ocean Altimeters TOPEXPoseidon Jason1 OSTMJason2 and Jason3 Version 51  PODAAC  JPL  NASA Search Type WebsiteDataSearch   Jet Propulsion LaboratoryCalifornia Institute of Technology PODAACFollow UsNavigation   Search Type WebsiteDataSearch HomeFind DataScience DisciplinesOcean Cryosphere Terrestrial Hydrosphere MeasurementsGlaciersIce SheetsGravityGravitational FieldOcean CirculationOcean Heat BudgetOcean Surface TopographyOcean TemperatureOcean WavesOcean WindsSalinityDensitySea IceSurface WaterMissionsADEOSIIAQUAAQUARIUSSACDCOWVRTEMPESTCYGNSSECCOGEOS3GHRSSTGRACEGRACEFOISSRAPIDSCAT JASON 1JASON 3MEASURESCCMPMEASURESMURMEASURESPRESWOTNASASSHNSCATOMGOPERAOSTMJASON 2QUIKSCATSMODESNPPSASSIESEASATSENTINEL6SMAPSPURSSWOTTERRATOPEXPOSEIDONAccess DataRESOURCESCloud Data UsersData ProvidersData UsersAboutABOUT UsABOUT PODAACData Use an

the system prompt is Extract metadata and entities details accurately from my requests.Metadata extraction for Indicators should be based on: [“Disasters”,“Human Dimensions”,“Food Availability”,“Health & Air Quality”,“Water Availability”, “Extreme Heat”,“Urban Flooding”,“Climate”] onlyMake sure the following fields follow the following regex patternSpatial_Resolution: ^(\d+(\.\d+)? [a-zA-Z]+|varies|N/A)$Temporal_Resolution: ^(\d+(\.\d+)? [a-zA-Z]+|N/A|varies|weekly|monthly|daily|yearly|varies-multiple datasets included|Daily < Weekly|Hourly < Daily|Weekly < Monthly|Monthly < Yearly|1 minute)$Temporal_Extent: ^(\d{4}-\d{2}-\d{2} to present|present|\d{4}-\d{2}-\d{2} (?:to|until) present|\d{4}-\d{2}-\d{2} to \d{4}-\d{2}-\d{2}|varies(?:- multiple datasets (?:included|available))?)$
The messages is [{'role': 'system', 'content': 'Extract metadata and entities details accurately from my requests.Metadata extraction for Indicators should be based on: [“Disasters”,“Human Dimensions”,“Food Avai

In [2]:
print("Before",metadata)
print("\n")
print("After",metadata)

Before {'id': 'chatcmpl-8qOSIRJvgu5lXELAk8mvgIk2AT6w4', 'object': 'chat.completion', 'created': 1707497754, 'model': 'gpt-3.5-turbo-0125', 'choices': [{'message': {'role': 'assistant', 'function_call': {'name': 'Environmental_Justice', 'arguments': '{"Indicators":"Climate","Description":"Global Mean Sea Level Trend from Integrated MultiMission Ocean Altimeters TOPEXPoseidon Jason1 OSTMJason2 and Jason3 Version 51","Geographic_Coverage":"GLOBAL","Format":"ASCII","Spatial_Resolution":"132 Decimal Degrees x 360 Decimal Degrees","Temporal_Resolution":"Weekly","Temporal_Extent":"1992Sep01 to Present","Latency":"N/A","Project":"Making Earth Science Data Records for Use in Research Environments MEaSUREs","Data_Visualization":true}'}}, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 4331, 'completion_tokens': 113, 'total_tokens': 4444}, 'system_fingerprint': 'fp_69829325d0'}


After {'id': 'chatcmpl-8qOSIRJvgu5lXELAk8mvgIk2AT6w4', 'object': 'chat.completion', 'created': 1707497754, 'mode

In [8]:
schema.__annotations__["Indicators"]

typing.Literal['Disasters', 'Human Dimensions', 'Food Availability', 'Health & Air Quality', 'Water Availability', 'Extreme Heat', 'Urban Flooding', 'Climate']